# Tidy Data 10/20/22

Goal: Put data into "tidy" (long) format

Timestamp | Room | Window Open | Temperature | RH 

In [1]:
import json
import datetime
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import math 
import sys
import os

# overall directory location 
root  = "/Users/julietnwagwuume-ezeoke/Library/CloudStorage/GoogleDrive-jnwagwu@stanford.edu/My Drive/UIL/windows/"
# add path to scripts 
sys.path.insert(0, os.path.join(root, 'analysis/scripts'))

In [146]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
from temprh_analysis import Temp_RH_Analysis

In [80]:
# TODO add these fx to temprh_analysis.py 

def str2dt(date):
    """Convert a string to a pandas timestamp object """
    return pd.to_datetime(date, format= '%Y, %m, %d, %H, %M' )

def get_htimes(df, date):
    with open('../constants/htimes.json') as f:    
        htimes = json.load(f)
    htimes_arr = [(str2dt(i["open"]), str2dt(i["close"])) for i in  htimes[date]]

    return htimes_arr

# Single DataFrame

In [153]:
a = Temp_RH_Analysis(root, "072522", ["Open", "Sometimes Open"] )
# TODO make file in constants with all times and variables 

In [154]:
a.trh_422a

,DateTime,Temp C,RH %
0,2022-07-18 22:30:00,26.090,50.099
1,2022-07-18 22:30:30,26.066,50.127
2,2022-07-18 22:31:00,26.042,50.571
3,2022-07-18 22:31:30,26.017,50.185
4,2022-07-18 22:32:00,25.993,50.309
...,...,...,...
18348,2022-07-25 07:24:00,22.901,53.616
18349,2022-07-25 07:24:30,22.925,53.557
18350,2022-07-25 07:25:00,22.901,53.491
18351,2022-07-25 07:25:30,22.925,53.557


* based on this, would just need to add a column about whether the window was open or closed based on h_times, and then group into one data set 

In [13]:
htimes = get_htimes(a, a.date)
htimes

[(Timestamp('2022-07-19 12:01:00'), Timestamp('2022-07-19 20:00:00')),
 (Timestamp('2022-07-20 07:50:00'), Timestamp('2022-07-20 12:30:00')),
 (Timestamp('2022-07-20 22:28:00'), Timestamp('2022-07-21 13:11:00')),
 (Timestamp('2022-07-21 15:06:00'), Timestamp('2022-07-21 17:28:00')),
 (Timestamp('2022-07-21 22:45:00'), Timestamp('2022-07-22 07:50:00')),
 (Timestamp('2022-07-22 11:07:00'), Timestamp('2022-07-22 16:00:00')),
 (Timestamp('2022-07-22 23:02:00'), Timestamp('2022-07-23 19:50:00')),
 (Timestamp('2022-07-24 13:22:00'), Timestamp('2022-07-24 19:33:00'))]

In [31]:
# create a column window open, with all 0 values (0 indicating closed)
df = a.trh_422a
df["Window Open"] = 0
# at these values in htime, change to 1 (indicating window open)
for htime_pair in htimes:
    mask = (df['DateTime'] > htime_pair [0]) & (df['DateTime'] <= htime_pair [1])
    df.loc[mask, "Window Open"]= 1

In [ ]:
def add_window_open_values(df, date):
    htimes = get_htimes(df, date)
    df["Window Open"] = 0
    # at these values in htime, change to 1 (indicating window open)
    for htime_pair in htimes:
        mask = (df['DateTime'] > htime_pair [0]) & (df['DateTime'] <= htime_pair [1])
        df.loc[mask, "Window Open"]= 1

In [34]:
df["Window Open"].mean()

0.462158775132131

In [159]:
# list of time stamps. for all time stamps, repeat for a and b. add column indicating if room a or room b 

with open('../constants/window_treatment.json') as f:    
    wt_file = json.load(f)
wt_file

{'072522': {'422a': 'Open',
  '422b': 'Sometimes Open',
  'cutoff_times': {'start': '2022, 07, 20, 07, 20',
   'end': '2022, 07, 24, 07, 20'}},
 '081622': {'422a': 'Sometimes Closed',
  '422b': 'Closed',
  'cutoff_times': {'start': '2022, 07, 27, 09, 00',
   'end': '2022, 08, 10, 20, 00'}},
 '092022': {'422a': 'Sometimes Closed',
  '422b': 'Closed',
  'cutoff_times': {'start': '2022, 09, 08, 08, 00',
   'end': '2022, 09, 11, 08, 00'}}}

In [194]:
df = a.trh_422a

In [195]:
start = str2dt(wt_file["072522"]["cutoff_times"]["start"])
end = str2dt(wt_file["072522"]["cutoff_times"]["end"])
filtered_df = df.loc[(df['DateTime'] >= start)
                     & (df['DateTime'] < end)]

In [196]:
filtered_df = df.loc[(df['DateTime'] >= start)
                     & (df['DateTime'] < end)].copy()

In [197]:
filtered_df.reset_index(inplace=True, drop=True)

In [198]:
filtered_df

,DateTime,Temp C,RH %
0,2022-07-20 07:20:00,23.189,51.268
1,2022-07-20 07:20:30,23.189,51.362
2,2022-07-20 07:21:00,23.189,51.488
3,2022-07-20 07:21:30,23.189,51.457
4,2022-07-20 07:22:00,23.165,51.517
...,...,...,...
11515,2022-07-24 07:17:30,23.309,52.009
11516,2022-07-24 07:18:00,23.285,52.069
11517,2022-07-24 07:18:30,23.285,52.100
11518,2022-07-24 07:19:00,23.309,51.915


In [199]:
filtered_df["Hello"] = 2

In [200]:
filtered_df

,DateTime,Temp C,RH %,Hello
0,2022-07-20 07:20:00,23.189,51.268,2
1,2022-07-20 07:20:30,23.189,51.362,2
2,2022-07-20 07:21:00,23.189,51.488,2
3,2022-07-20 07:21:30,23.189,51.457,2
4,2022-07-20 07:22:00,23.165,51.517,2
...,...,...,...,...
11515,2022-07-24 07:17:30,23.309,52.009,2
11516,2022-07-24 07:18:00,23.285,52.069,2
11517,2022-07-24 07:18:30,23.285,52.100,2
11518,2022-07-24 07:19:00,23.309,51.915,2


In [201]:
del filtered_df

# All DataFrames

In [221]:
def add_window_open_values(df, date, treatment):
    # print(treatment)
    htimes = get_htimes(df, date)
    df["Window Open"] = 0 if "Open" in treatment else 0 # could change this in future if needed, but treatment currently does not affect validty of this function 
    # at these values in htime, change to 1 (indicating window open)
    for htime_pair in htimes:
        mask = (df['DateTime'] > htime_pair [0]) & (df['DateTime'] <= htime_pair [1])
        df.loc[mask, "Window Open"] = 1 if "Open" in treatment else 1
    return df

In [151]:
# list of time stamps. for all time stamps, repeat for a and b. add column indicating if room a or room b 

with open('../constants/window_treatment.json') as f:    
    wt_file = json.load(f)
wt_file

{'072522': {'422a': 'Open',
  '422b': 'Sometimes Open',
  'cutoff_times': {'start': '2022, 07, 20, 07, 20, 30',
   'end': '2022, 07, 24, 07, 20, 30'}},
 '081622': {'422a': 'Sometimes Closed',
  '422b': 'Closed',
  'cutoff_times': {'start': '2022, 07, 27, 09, 00, 30',
   'end': '2022, 08, 10, 20, 00, 00'}},
 '092022': {'422a': 'Sometimes Closed',
  '422b': 'Closed',
  'cutoff_times': {'start': '2022, 09, 08, 08, 00, 30',
   'end': '2022, 09, 11, 08, 00, 00'}}}

In [181]:
rooms = [room for room in wt_file[date] if "422" in room]

In [182]:
rooms

['422a', '422b']

In [237]:
all_data_list = []

for date in wt_file.keys(): # for checking -- list(wt_file.keys())[0:1]
    rooms = [room for room in wt_file[date] if "422" in room]
    for room in rooms:
        date_data = Temp_RH_Analysis(root, date, "_" )

        # assign room values and get appropriate df for the room 
        if room == "422a":
            df =  date_data.trh_422a
            df["Room"] = 0
            print(room)
        elif room == "422b":
            df = date_data.trh_422b
            df["Room"] = 1
            print(room)
        
        # apply cutoff data 
        start = str2dt(wt_file[date]["cutoff_times"]["start"])
        end = str2dt(wt_file[date]["cutoff_times"]["end"])
        filtered_df = df.loc[(df['DateTime'] >= start)
                            & (df['DateTime'] <= end)].copy()
        filtered_df.reset_index(inplace=True, drop=True)

        print(filtered_df["DateTime"].tail(1))
        # print("middle")

        if wt_file[date][room] == "Open":
            # window open -> 1
            filtered_df["Window Open"] = 1
            print(f"-> {wt_file[date][room]}")
        elif "Sometimes" in  wt_file[date][room]:
            # window open -> function 
            filtered_df = add_window_open_values(filtered_df, date, wt_file[date][room] )
            print(f"-> {wt_file[date][room]}")
        elif wt_file[date][room] == "Closed":
            # window open -> 0
            filtered_df["Window Open"] = 0
            print(f"-> {wt_file[date][room]}")

        print(filtered_df["DateTime"].tail(1))
        
        # add this data frame to the list 
        all_data_list.append(filtered_df)

422a
11520   2022-07-24 07:20:00
Name: DateTime, dtype: datetime64[ns]
-> Open
11520   2022-07-24 07:20:00
Name: DateTime, dtype: datetime64[ns]
422b
11520   2022-07-24 07:20:00
Name: DateTime, dtype: datetime64[ns]
-> Sometimes Open
11520   2022-07-24 07:20:00
Name: DateTime, dtype: datetime64[ns]
422a
41640   2022-08-10 20:00:00
Name: DateTime, dtype: datetime64[ns]
-> Sometimes Closed
41640   2022-08-10 20:00:00
Name: DateTime, dtype: datetime64[ns]
422b
41640   2022-08-10 20:00:00
Name: DateTime, dtype: datetime64[ns]
-> Closed
41640   2022-08-10 20:00:00
Name: DateTime, dtype: datetime64[ns]
422a
8640   2022-09-11 08:00:00
Name: DateTime, dtype: datetime64[ns]
-> Sometimes Closed
8640   2022-09-11 08:00:00
Name: DateTime, dtype: datetime64[ns]
422b
8640   2022-09-11 08:00:00
Name: DateTime, dtype: datetime64[ns]
-> Closed
8640   2022-09-11 08:00:00
Name: DateTime, dtype: datetime64[ns]


In [223]:
all_data_list

[                 DateTime  Temp C    RH %  Room  Window Open
 0     2022-07-20 07:20:00  23.189  51.268     0            1
 1     2022-07-20 07:20:30  23.189  51.362     0            1
 2     2022-07-20 07:21:00  23.189  51.488     0            1
 3     2022-07-20 07:21:30  23.189  51.457     0            1
 4     2022-07-20 07:22:00  23.165  51.517     0            1
 ...                   ...     ...     ...   ...          ...
 11516 2022-07-24 07:18:00  23.285  52.069     0            1
 11517 2022-07-24 07:18:30  23.285  52.100     0            1
 11518 2022-07-24 07:19:00  23.309  51.915     0            1
 11519 2022-07-24 07:19:30  23.309  51.852     0            1
 11520 2022-07-24 07:20:00  23.309  51.883     0            1
 
 [11521 rows x 5 columns],
                  DateTime  Temp C    RH %  Room  Window Open
 0     2022-07-18 22:30:00  25.847  52.547     1            0
 1     2022-07-18 22:30:30  25.822  52.576     1            0
 2     2022-07-18 22:31:00  25.798  52.69

In [238]:
for ix, data in enumerate(all_data_list):
    print(ix, data["Window Open"].mean() )

0 1.0
1 0.5886641784567311
2 0.9482481208424389
3 0.0
4 0.7783821316977202
5 0.0


In [226]:
all_data_list[4]["DateTime"].tail(2)

34909   2022-09-20 10:39:30
34910   2022-09-20 10:40:00
Name: DateTime, dtype: datetime64[ns]

In [206]:
all_data_list[4]["Window Open"]

0        0
1        0
2        0
3        0
4        0
        ..
34906    0
34907    0
34908    0
34909    0
34910    0
Name: Window Open, Length: 34911, dtype: int64

In [148]:
import plotly.express as px

In [239]:
# fig = make_subplots()
# fig.add_trace(go.Scatter())
fig = px.line(all_data_list[4], x="DateTime", y="Window Open" )
fig.show()

In [243]:
# add to total df  
all_data_df = pd.concat(all_data_list, ignore_index=True)

In [244]:
all_data_df

,DateTime,Temp C,RH %,Room,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,1
1,2022-07-20 07:20:30,23.189,51.362,0,1
2,2022-07-20 07:21:00,23.189,51.488,0,1
3,2022-07-20 07:21:30,23.189,51.457,0,1
4,2022-07-20 07:22:00,23.165,51.517,0,1
...,...,...,...,...,...
123601,2022-09-11 07:58:00,29.196,50.861,1,0
123602,2022-09-11 07:58:30,29.196,50.861,1,0
123603,2022-09-11 07:59:00,29.196,50.796,1,0
123604,2022-09-11 07:59:30,29.196,50.796,1,0


In [247]:
all_data_df["Window Open"].mean() 

0.5219406824911412

In [246]:
all_data_df["Room"].mean() # should be 0.5 -> same amount of data collected in Rooms A and B 

0.5

In [240]:
all_data_df.loc[(all_data_df['DateTime'] >= "2022-09-01")]

,DateTime,Temp C,RH %,Room,Window Open
0,2022-09-08 07:45:00,29.446,46.194,0,0
1,2022-09-08 07:45:30,29.446,46.259,0,0
2,2022-09-08 07:46:00,29.446,46.292,0,0
3,2022-09-08 07:46:30,29.421,46.289,0,0
4,2022-09-08 07:47:00,29.421,46.289,0,0
...,...,...,...,...,...
8636,2022-09-11 07:58:00,29.196,50.861,1,0
8637,2022-09-11 07:58:30,29.196,50.861,1,0
8638,2022-09-11 07:59:00,29.196,50.796,1,0
8639,2022-09-11 07:59:30,29.196,50.796,1,0


## save as csv

In [81]:
all_data_df.to_csv("../constants/tidydata_102022.csv")